# 데이터 전처리

In [25]:
import cv2
import os
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import ConvLSTM2D, BatchNormalization, MaxPooling3D, Flatten, Dense
from tqdm import tqdm
# 데이터 경로 설정
bread_path = r'G:\내 드라이브\Sign_Remaster\Sign_Language_Remaster\data\bread_house'
hello_path = r'G:\내 드라이브\Sign_Remaster\Sign_Language_Remaster\data\hello'

# 함수: 영상 읽기 및 전처리
def read_and_preprocess_data(folder_path):
    data = []
    for filename in os.listdir(folder_path):
        if filename.endswith('.avi'):
            filepath = os.path.join(folder_path, filename)
            cap = cv2.VideoCapture(filepath)
            frames = []
            while True:
                ret, frame = cap.read()
                if not ret:
                    break
                # 영상 프레임을 128x128 크기로 resize하고 RGB로 변환
                frame = cv2.resize(frame, (128, 128))
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                frames.append(frame)
            cap.release()
            data.extend(frames)
    return data

# 데이터 읽기 및 전처리
bread_data = read_and_preprocess_data(bread_path)
hello_data = read_and_preprocess_data(hello_path)

# 시퀀스 길이 정의
sequence_length = 30  # 예시로 30프레임으로 고정

# 데이터를 시퀀스 단위로 이어 붙이기
def create_sequences(data, sequence_length):
    sequences = []
    # num_sequences = len(data) // sequence_length
    num_sequences = len(data) // sequence_length
    
    for i in tqdm(range(num_sequences)):
        sequence = data[i * sequence_length : (i + 1) * sequence_length]
        sequences.append(sequence)
    
    return sequences

# 시퀀스 생성
bread_sequences = create_sequences(bread_data, sequence_length)
hello_sequences = create_sequences(hello_data, sequence_length)

# 텐서로 변환 (샘플 수, 프레임 수, 이미지 높이, 이미지 너비, 채널 수)
X_bread = np.array(bread_sequences).reshape(-1, sequence_length, 128, 128, 3)
X_hello = np.array(hello_sequences).reshape(-1, sequence_length, 128, 128, 3)

# 레이블 설정
y_bread = np.zeros(len(X_bread))  # bread: 0
y_hello = np.ones(len(X_hello))   # hello: 1

# 데이터 결합
X = np.concatenate((X_bread, X_hello))
y = np.concatenate((y_bread, y_hello))



100%|██████████| 124/124 [00:00<?, ?it/s]


In [26]:
X.shape

(169, 30, 128, 128, 3)

In [27]:
y.shape

(169,)

# 모델 학습

In [31]:

# 모델 구축
model = Sequential([
    ConvLSTM2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same', return_sequences=True, input_shape=(sequence_length, 128, 128, 3)),
    BatchNormalization(),
    MaxPooling3D(pool_size=(1, 2, 2)),
    ConvLSTM2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same', return_sequences=True),
    BatchNormalization(),
    MaxPooling3D(pool_size=(1, 2, 2)),
    Flatten(),
    Dense(256, activation='relu'),
    Dense(1, activation='sigmoid')  # 이진 분류를 위한 출력 레이어
])

# 모델 컴파일
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 모델 학습
model.fit(X, y, epochs=10, batch_size=16, validation_split=0.2)

KeyboardInterrupt: 

# 테스트

In [ ]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

# 저장된 모델 불러오기
model = load_model('path/to/your/model.h5')  # 학습된 모델의 경로를 설정해야 합니다.

# 모델이 예측하는 클래스의 이름 설정
class_names = ['bread_house', 'hello']  # 각 클래스에 해당하는 이름을 설정해야 합니다.

# 웹캠 또는 비디오 스트림 열기
cap = cv2.VideoCapture(0)  # 웹캠을 사용하려면 0을 입력하거나, 비디오 파일 경로를 입력하세요.

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # 프레임 전처리 (크기 조정 등)
    processed_frame = cv2.resize(frame, (128, 128))
    processed_frame = cv2.cvtColor(processed_frame, cv2.COLOR_BGR2RGB)
    processed_frame = np.expand_dims(processed_frame, axis=0)  # 모델에 입력하기 위해 차원 확장

    # 모델로 예측
    prediction = model.predict(processed_frame)
    predicted_class = np.argmax(prediction)

    # 예측 결과를 화면에 표시
    text = f"Predicted: {class_names[predicted_class]}"
    cv2.putText(frame, text, (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    
    # 화면에 영상 표시
    cv2.imshow('Real-time Prediction', frame)

    # 'q'를 누르면 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 작업 완료 후 해제
cap.release()
cv2.destroyAllWindows()
